# Fisher 判别（LDA）大作业 - 运行版

这是为你准备的**完整 Jupyter Notebook 模板**，你可以直接下载并在本地运行（推荐使用 Anaconda + JupyterLab/Notebook）。

**包含内容**：

- 手写实现 Fisher 判别 / 多类 LDA（含数值稳定性处理）
- 在 Iris（内置）与 Sonar（UCI）上做实验（Stratified k-fold CV）
- 绘制：Accuracy vs 投影维数、投影散点图、ROC（Sonar）等
- 与 sklearn 的 LinearDiscriminantAnalysis 做对比

**如何使用**：

1. 下载本 notebook（页面下方提供下载链接）。
2. 在能联网的环境中运行（Sonar 数据需要从 UCI 下载，Notebook 中有自动下载代码；若公司/校园网络限制，可手动下载 sonar 数据并放到 notebooks 同目录下）。
3. 按顺序运行每个 Cell，查看并复制输出结果到你的报告中。

如果你不希望/不能联网，也可以只运行 Iris 部分（Iris 数据内置）。

---

下面是 notebook 的内容（已包含注释，便于你“零代码”运行）。

In [1]:
# 导入常用库（运行前请确保已安装：numpy, scipy, scikit-learn, matplotlib, pandas）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import time
plt.rcParams['figure.figsize'] = (6,4)
print('Libraries imported. Numpy version:', np.__version__)

Libraries imported. Numpy version: 1.26.4


## 载入数据集

- **Iris**：直接使用 sklearn 内置加载函数。
- **Sonar**：UCI 上的 `sonar.all-data` 文件。下面给出两种方式：
  1. 自动下载（需要联网）。
  2. 如果无法联网，请手动从 UCI 页面下载 `sonar.all-data`，并将其放到本 notebook 的同一目录，命名为 `sonar.all-data`。

**UCI Sonar 数据集 URL（页面）**: https://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)

（Notebook 中包含自动下载代码；若你本地能联网，运行该 Cell 会尝试下载并解析 Sonar）

In [2]:
# Load Iris (built-in)
iris = load_iris()
X_iris = iris.data
y_iris = iris.target
print('Iris shapes:', X_iris.shape, y_iris.shape)
print('Iris class names:', iris.target_names)

Iris shapes: (150, 4) (150,)
Iris class names: ['setosa' 'versicolor' 'virginica']


In [ ]:
# Load Sonar: try to download; if fails, try to read local file 'sonar.all-data'
import os
sonar_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data'
sonar_local = 'sonar.all-data'

def load_sonar():
    # Try to read local file first
    if os.path.exists(sonar_local):
        print('Loading Sonar from local file:', sonar_local)
        df = pd.read_csv(sonar_local, header=None)
    else:
        # Try to download (this requires network access)
        try:
            import requests
            print('Downloading Sonar from UCI...')
            r = requests.get(sonar_url, timeout=30)
            r.raise_for_status()
            open(sonar_local, 'wb').write(r.content)
            df = pd.read_csv(sonar_local, header=None)
            print('Downloaded and saved to', sonar_local)
        except Exception as e:
            print('自动下载失败（可能因网络限制）。请手动从 UCI 下载 sonar.all-data 并放到当前目录，或确保本环境能联网。')
            raise e
    # Last column is label 'R' or 'M'
    X = df.iloc[:, :-1].values.astype(float)
    y_raw = df.iloc[:, -1].values
    y = np.array([1 if lab.strip()=='M' else 0 for lab in y_raw])  # M = mine (1), R = rock (0)
    return X, y

# Note: In this execution environment internet may be disabled; if load_sonar() fails here, it's fine.
try:
    X_sonar, y_sonar = load_sonar()
    print('Sonar shapes:', X_sonar.shape, y_sonar.shape)
except Exception as e:
    print('Sonar 数据未能加载到这个环境。不要担心：在你自己的电脑上运行此 notebook（有网络或手动放置文件）即可。')

## 手写 LDA（多类与二类通用实现）

下面的实现包含：

- 计算类内散度矩阵 $S_w$ 和类间散度矩阵 $S_b$。

- 对 $S_w^{-1} S_b$ 求特征值/特征向量，得到前 k 个判别向量。

- 数值稳定处理：在求逆前添加小正则项 $\\epsilon I$。

实现后的函数 `lda_fit` 返回投影矩阵 `W`（每列为一个判别方向），`lda_project` 将数据投影到这些方向上。

In [ ]:
import numpy.linalg as la

def lda_fit(X, y, n_components=None, reg=1e-6):
    '''
    手写 LDA：
    X: (n_samples, n_features)
    y: (n_samples,)  int labels: 0..c-1
    n_components: desired number of discriminant directions (<= min(c-1, n_features))
    reg: regularization added to S_w for numerical stability
    returns W: (n_features, n_components), eigenvalues
    '''
    classes = np.unique(y)
    n_features = X.shape[1]
    if n_components is None:
        n_components = min(len(classes)-1, n_features)
    overall_mean = np.mean(X, axis=0)
    # within-class scatter
    S_w = np.zeros((n_features, n_features))
    S_b = np.zeros((n_features, n_features))
    for cl in classes:
        Xc = X[y==cl]
        mean_c = np.mean(Xc, axis=0)
        # within
        Xc_centered = Xc - mean_c
        S_w += Xc_centered.T.dot(Xc_centered)
        # between
        n_c = Xc.shape[0]
        mean_diff = (mean_c - overall_mean).reshape(n_features,1)
        S_b += n_c * (mean_diff).dot(mean_diff.T)
    # regularize S_w
    S_w_reg = S_w + reg * np.eye(n_features)
    # solve generalized eigenvalue problem: S_b v = lambda S_w v  -> equivalent to inv(S_w) S_b v = lambda v
    # compute matrix for eig
    mat = la.inv(S_w_reg).dot(S_b)
    eigvals, eigvecs = la.eig(mat)
    # sort by eigenvalue magnitude descending
    idx = np.argsort(-np.abs(eigvals.real))
    eigvals = eigvals.real[idx]
    eigvecs = eigvecs.real[:, idx]
    W = eigvecs[:, :n_components]
    return W, eigvals[:n_components]

def lda_project(X, W):
    return X.dot(W)

# Quick sanity check on Iris
W_iris, eigs_iris = lda_fit(X_iris, y_iris, n_components=2)
print('W_iris shape:', W_iris.shape)
print('Top eigenvalues (iris):', eigs_iris)

## 在投影空间上的分类与交叉验证（Stratified k-fold）

这里使用**最近类均值（Nearest Centroid）**作为投影空间的简单分类器：

1. 在训练集上拟合 LDA（得到 W）。
2. 将训练集投影并计算各类投影均值。
3. 将测试集投影，按最近均值分配标签。

下面实现 `evaluate_lda_cv`：对不同投影维数 `d_list` 做性能曲线（accuracy 平均与标准差）。

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class NearestCentroidInProjectedSpace(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.centroids_ = None
    def fit(self, X_proj, y):
        classes = np.unique(y)
        self.classes_ = classes
        self.centroids_ = np.vstack([X_proj[y==c].mean(axis=0) for c in classes])
        return self
    def predict(self, X_proj):
        # compute distances to centroids
        dists = np.linalg.norm(X_proj[:, None, :] - self.centroids_[None, :, :], axis=2)
        idx = np.argmin(dists, axis=1)
        return self.classes_[idx]

def evaluate_lda_cv(X, y, d_list, n_splits=5, random_state=0, reg=1e-6):
    """
    For each d in d_list, perform StratifiedKFold CV, fit LDA on training fold, fit nearest-centroid on projected training data,
    evaluate on test fold. Return mean acc and std acc for each d.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    results = {d: [] for d in d_list}
    for train_idx, test_idx in skf.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        # standardize based on training data
        scaler = StandardScaler().fit(X_train)
        X_train_s = scaler.transform(X_train)
        X_test_s = scaler.transform(X_test)
        for d in d_list:
            W, _ = lda_fit(X_train_s, y_train, n_components=d, reg=reg)
            Xtr_proj = lda_project(X_train_s, W)
            Xte_proj = lda_project(X_test_s, W)
            clf = NearestCentroidInProjectedSpace().fit(Xtr_proj, y_train)
            y_pred = clf.predict(Xte_proj)
            results[d].append(accuracy_score(y_test, y_pred))
    # compute mean and std
    mean_std = {d: (np.mean(results[d]), np.std(results[d])) for d in d_list}
    return mean_std, results

# Example run on Iris (d candidates: 1,2)
d_list_iris = [1,2]
mean_std_iris, raw_iris = evaluate_lda_cv(X_iris, y_iris, d_list_iris, n_splits=10, random_state=42)
print('Iris results (mean, std):', mean_std_iris)

## 对 Sonar 的评价

如果你已经在本地成功加载 Sonar（上面 `load_sonar()` 的代码成功），可以运行下面的 Cell：

- 选择合适的 `d_list_sonar`（例如 `[1,2,5,10,20,30,40,50,60]`）。
- 使用 `n_splits=5` 的 StratifiedKFold。

如果 Sonar 尚未加载（因网络或未手动放置文件），跳过此 Cell，先做 Iris。

In [1]:
# Only run if X_sonar exists
try:
    X_sonar
    d_list_sonar = [1,2,5,10,20,30,40,50,60]
    mean_std_sonar, raw_sonar = evaluate_lda_cv(X_sonar, y_sonar, d_list_sonar, n_splits=5, random_state=42, reg=1e-4)
    print('Sonar results (d -> (mean,std)):')
    for d in d_list_sonar:
        mn, sd = mean_std_sonar[d]
        print(f'd={d}: mean acc={mn:.4f}, std={sd:.4f}')
except NameError:
    print('Sonar 未在当前环境加载；如要运行 Sonar 部分，请确保 sonar.all-data 在同目录或环境可联网以自动下载。')

Sonar 未在当前环境加载；如要运行 Sonar 部分，请确保 sonar.all-data 在同目录或环境可联网以自动下载。


## 绘图：Accuracy vs 投影维数 与 投影散点图（Iris）

下面画出 Iris 的 accuracy 随 d 变化（d=1,2）。并展示在前 2 个判别向量下的投影散点图。

In [ ]:
# Accuracy vs d (Iris)
d_vals = sorted(mean_std_iris.keys())
means = [mean_std_iris[d][0] for d in d_vals]
stds = [mean_std_iris[d][1] for d in d_vals]

plt.errorbar(d_vals, means, yerr=stds, marker='o', linestyle='-')
plt.xlabel('投影维数 d')
plt.ylabel('Accuracy (mean ± std)')
plt.title('Iris: Accuracy vs projection dimension (handwritten LDA)')
plt.xticks(d_vals)
plt.grid(True)
plt.show()

# Scatter in top 2 discriminants
W, _ = lda_fit(StandardScaler().fit_transform(X_iris), y_iris, n_components=2)
X_iris_proj = lda_project(StandardScaler().fit_transform(X_iris), W)
plt.figure(figsize=(6,5))
for lab, name in enumerate(iris.target_names):
    plt.scatter(X_iris_proj[y_iris==lab,0], X_iris_proj[y_iris==lab,1], label=name, alpha=0.8)
plt.xlabel('LD1')
plt.ylabel('LD2')
plt.title('Iris projected to first 2 LDA components')
plt.legend()
plt.grid(True)
plt.show()

## 与 scikit-learn 的 LDA 做对比

使用 `sklearn.discriminant_analysis.LinearDiscriminantAnalysis` 在相同 CV 设置下进行对比，检查准确率与手写实现是否一致。

In [ ]:
def evaluate_sklearn_lda_cv(X, y, d_list, n_splits=5, random_state=0, solver='svd'):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    results = {d: [] for d in d_list}
    for train_idx, test_idx in skf.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        scaler = StandardScaler().fit(X_train)
        X_train_s = scaler.transform(X_train)
        X_test_s = scaler.transform(X_test)
        for d in d_list:
            lda = LinearDiscriminantAnalysis(n_components=d, solver=solver)
            lda.fit(X_train_s, y_train)
            y_pred = lda.predict(X_test_s)
            results[d].append(accuracy_score(y_test, y_pred))
    mean_std = {d: (np.mean(results[d]), np.std(results[d])) for d in d_list}
    return mean_std, results

# Compare on Iris
d_list_iris = [1,2]
mean_std_sklearn_iris, _ = evaluate_sklearn_lda_cv(X_iris, y_iris, d_list_iris, n_splits=10, random_state=42)
print('sklearn LDA (Iris):', mean_std_sklearn_iris)
print('handwritten LDA (Iris):', mean_std_iris)

## 保存结果与报告写作提示

- 你可以将输出的图表另存为 PNG（`plt.savefig('figure.png', dpi=300)`）并插入报告。
- 在报告中写明：使用 Stratified 10-fold（Iris）和 Stratified 5-fold（Sonar），手写 LDA 与 sklearn 的对比结果（给出均值 ± 标准差）。
- 报告附录中贴上关键代码片段（Notebook 中的 cells 就足够）。

---

**现在我已经把这个 Notebook 生成为文件**，你可以下载并在本地运行。若你需要，我可以把 Notebook 中的某些 cell 进一步简化，让你运行时只需按顺序按 Run（无需阅读代码）并导出结果表格。